## CS584_Homework 3_Part 1

**Pre-Requirment:**<br>
1. nltk
2. collections

### Data Pre-processing

In [1]:
import re
import numpy as np
from nltk import bigrams
from collections import defaultdict, Counter

In [2]:
#File path
file = 'a3-data/train.txt'

# Clean Data
# Input: file path
def clean_text(filepath):
    
    f = open('a3-data/train.txt',"r", encoding='utf-8')
    lines = f.readlines()
    output = []
    for i in range(0,lines.__len__(),1):
        for word in lines[i].split():
            word = word.strip(' ')
            word = re.sub(r"[-()\"#/@;:{}'+=|.!?,]", "", word)
            output.append(word)
    
    return output

train_word = clean_text(file)
print(len(train_word))
#print(train_word[0:100])

887521


### Build bi-Gram Language Model

In [3]:
# Create set of all unique words,
import sets

gram_1 = set(train_word)
print(len(gram_1))

# Visualize sets results
gram1_iter = iter(gram_1)
print([gram1_iter.__next__() for i in range(20)])

9942
['', 'hardware', 'squeeze', 'lungcancer', 'prevailed', 'thing', 'he', 'finances', 'thus', 'score', 'test', 'jerry', 'verge', 'marous', 'requirement', 'actions', 'plane', 'agree', 'poughkeepsie', 'specialized']


In [4]:
# Construct word pairs

word_pairs = [(train_word[i], train_word[i+1]) for i in range(len(train_word)-1)]
print(len(word_pairs))

# gram_2 = set(word_pairs)
# print(len(gram_2))
# # Print 10 elements from gram2
# gram2_iter = iter(gram_2)
# print([gram2_iter.__next__() for i in range(10)])

887520


In [5]:
"""
Calculate the frequency for each single word.
Results will be used in the following step.
"""
gram1_freq = dict()

# Populate 1-gram dictionary
for word in train_word:
    if word in gram1_freq:
        gram1_freq[word] += 1
    else:
        gram1_freq[word] = 1 

# Turn into a list of (word, count) sorted by count from most to least
#gram1_freq = sorted(gram1_freq.items(), key = lambda x: x[1], reverse = True)

# Print top 20 most frequent words
#print(gram1_freq[:20])

In [6]:
"""
Calculate the frequency for word pair.
"""
gram2_freq = dict()

# Populate 2-gram dictionary
for i in range(len(train_word)-1):
    key = (train_word[i], train_word[i+1])
    if key in gram2_freq:
        gram2_freq[key] += 1
    else:
        gram2_freq[key] = 1

# Print top 20 most frequent word pairs
#print(gram2_freq[:20])

In [28]:
"""
Calculate the probability 
"""
gram2_prob = dict()
for key, value in gram2_freq.items():
    if key[0] in gram1_freq:
        gram2_prob[key] = (value / gram1_freq[key[0]])
    if key[0] not in gram1_freq:
        gram2_prob[key] = 0
        
gram2_prob_print = sorted(gram2_prob.items(), key = lambda x: x[1], reverse = True)
# Print some probability results
print(gram2_prob_print[500:520])

[(('spin', 'off'), 0.6842105263157895), (('refining', 'and'), 0.6842105263157895), (('signs', 'of'), 0.6833333333333333), (('philip', 'morris'), 0.6833333333333333), (('salomon', 'brothers'), 0.6829268292682927), (('est', 'on'), 0.6818181818181818), (('bear', 'stearns'), 0.6774193548387096), (('does', 'nt'), 0.6685714285714286), (('seagate', 'technology'), 0.6666666666666666), (('ed', '<unk>'), 0.6666666666666666), (('tom', '<unk>'), 0.6666666666666666), (('relied', 'on'), 0.6666666666666666), (('johns', 'hopkins'), 0.6666666666666666), (('threatens', 'to'), 0.6666666666666666), (('patrick', '<unk>'), 0.6666666666666666), (('aspect', 'of'), 0.6666666666666666), (('franc', 'note'), 0.6666666666666666), (('responsive', 'to'), 0.6666666666666666), (('prescription', 'drugs'), 0.6666666666666666), (('exodus', 'of'), 0.6666666666666666)]


### Good Turing smoothing

In [10]:
# Calculate the unseen word
def get_no_of_unseen_bigrams(bigrams_dict):
    seen_bigrams = 0
    for bi in bigrams_dict:
        seen_bigrams += bigrams_dict[bi]
    unseen_bigrams = len(bigrams_dict) * len(bigrams_dict) - seen_bigrams
    return unseen_bigrams

#Assume seen once bigrams = unseen bigrams
def get_no_of_seen_once_bigrams(bigrams_dict):
    counts = 0
    for bi in bigrams_dict:
        if(bigrams_dict.get(bi) == 1):
            counts = counts + 1
    return counts

#Compute revised Good-Turing counts
def revise_counts(bigrams_dict):
    no_of_seen_once_bigrams = get_no_of_seen_once_bigrams(bigrams_dict)
    no_of_unseen_bigrams = get_no_of_unseen_bigrams(bigrams_dict)
    adjusted_unseen_bigrams = no_of_seen_once_bigrams / no_of_unseen_bigrams
    new_counts = [adjusted_unseen_bigrams]
    prev_count = no_of_seen_once_bigrams
    for i in range(2,7):
        count = 0
        for bi in bigrams_dict:
            if( bigrams_dict.get(bi) == i):
                count = count + 1
        good_turing_count = i * count / prev_count
        prev_count = count
        new_counts.append(good_turing_count) 
    return new_counts

# implement the good_turing
def good_turing_bigrams_dict_count2(bigrams_dict):
    new_counts = revise_counts(bigrams_dict)
    for i in range (1,6):
        for bi in bigrams_dict:
            if (bigrams_dict.get(bi) == i):
                bigrams_dict[bi] = new_counts[i]
    bigrams_dict[('unseen', 'unseen')] = new_counts[0]
    return bigrams_dict

good_turing_bigrams_dict_count2(gram2_freq)

{('aer', 'banknote'): 0.3885869135424959,
 ('banknote', 'berlitz'): 0.3885869135424959,
 ('berlitz', 'calloway'): 0.3885869135424959,
 ('calloway', 'centrust'): 0.3885869135424959,
 ('centrust', 'cluett'): 0.3885869135424959,
 ('cluett', 'fromstein'): 0.3885869135424959,
 ('fromstein', 'gitano'): 0.3885869135424959,
 ('gitano', 'guterman'): 0.3885869135424959,
 ('guterman', 'hydroquebec'): 0.3885869135424959,
 ('hydroquebec', 'ipo'): 0.3885869135424959,
 ('ipo', 'kia'): 0.3885869135424959,
 ('kia', 'memotec'): 0.3885869135424959,
 ('memotec', 'mlx'): 0.3885869135424959,
 ('mlx', 'nahb'): 0.3885869135424959,
 ('nahb', 'punts'): 0.3885869135424959,
 ('punts', 'rake'): 0.3885869135424959,
 ('rake', 'regatta'): 0.3885869135424959,
 ('regatta', 'rubens'): 0.3885869135424959,
 ('rubens', 'sim'): 0.3885869135424959,
 ('sim', 'snackfood'): 0.3885869135424959,
 ('snackfood', 'ssangyong'): 0.3885869135424959,
 ('ssangyong', 'swapo'): 0.3885869135424959,
 ('swapo', 'wachter'): 0.3885869135424959,

### Kneser-Ney Smoothing

In [11]:
from nltk.lm.api import Smoothing

class KneserNey(Smoothing):
    """Kneser-Ney Smoothing."""

    def __init__(self, vocabulary, counter, discount=0.1, **kwargs):
        super().__init__(vocabulary, counter, **kwargs)
        self.discount = discount

    def unigram_score(self, word):
        return 1.0 / len(self.vocab)

    def alpha_gamma(self, word, context):
        prefix_counts = self.counts[context]
        prefix_total_ngrams = prefix_counts.N()
        alpha = max(prefix_counts[word] - self.discount, 0.0) / prefix_total_ngrams
        gamma = (
            self.discount * _count_non_zero_vals(prefix_counts) / prefix_total_ngrams
        )
        return alpha, gamma


Tried to implement Knerser-Ney based on mathematical equation in homework instruction by using Python and browsed some offical documents. The code is shown in above block. However, it failled. The bug is about data type. Therefore, we finished this step by using nltk library first.  

In [12]:
import nltk
from nltk import ngrams
freq_dist = nltk.FreqDist(ngrams(gram_1, 3,))
knerser_ney = nltk.KneserNeyProbDist(freq_dist)
[(i, knerser_ney.prob(i)) for i in knerser_ney.samples()]

[(('', 'hardware', 'squeeze'), 0.25),
 (('hardware', 'squeeze', 'lungcancer'), 0.25),
 (('squeeze', 'lungcancer', 'prevailed'), 0.25),
 (('lungcancer', 'prevailed', 'thing'), 0.25),
 (('prevailed', 'thing', 'he'), 0.25),
 (('thing', 'he', 'finances'), 0.25),
 (('he', 'finances', 'thus'), 0.25),
 (('finances', 'thus', 'score'), 0.25),
 (('thus', 'score', 'test'), 0.25),
 (('score', 'test', 'jerry'), 0.25),
 (('test', 'jerry', 'verge'), 0.25),
 (('jerry', 'verge', 'marous'), 0.25),
 (('verge', 'marous', 'requirement'), 0.25),
 (('marous', 'requirement', 'actions'), 0.25),
 (('requirement', 'actions', 'plane'), 0.25),
 (('actions', 'plane', 'agree'), 0.25),
 (('plane', 'agree', 'poughkeepsie'), 0.25),
 (('agree', 'poughkeepsie', 'specialized'), 0.25),
 (('poughkeepsie', 'specialized', 'controls'), 0.25),
 (('specialized', 'controls', 'dunkin'), 0.25),
 (('controls', 'dunkin', 'stein'), 0.25),
 (('dunkin', 'stein', 'item'), 0.25),
 (('stein', 'item', 'any'), 0.25),
 (('item', 'any', 'burni

### Preplexity

In [19]:
import jieba

def perplexity(sentence, uni_gram_dict, bi_gram_dict):
    sentence_cut = list(jieba.cut(sentence))
    V = len(uni_gram_dict)
    sentence_len = len(sentence_cut)
    p=1
    k=0.5  
    for i in range(sentence_len-1):
        two_word = "".join(sentence_cut[i:i+2])
        p *=(bi_gram_dict.get(two_word,0)+k)/(uni_gram_dict.get(sentence_cut[i],0)+k*V)

    return pow(1/p, 1/sentence_len)

Based on the definition,  we implemented preplexity calculation process. However, because of the problem in bigram modeling, we didn't get the final perplexity results. 

### Prediction

In [44]:
test_file = open('a3-data/input.txt',"r", encoding='utf-8')
lines = test_file.readlines()
test_line = lines[:30]
i = 1
for line in test_line:
    line = re.sub(r"[-()\"#/@;:{}'+=_|.!?,]", "", line)
    word = line.split()
    test_word = word[-1]
    results = dict()
    for key, value in gram2_prob.items():
        if test_word in key[0]:
            results[key] = value
        else:
            continue
    pred =sorted(results, key = lambda x:results[x])[-1]
    print(i, "The next word of", test_word, "is :", pred[1] )
    i += 1

1 The next word of fall is : from
2 The next word of failed is : to
3 The next word of floor is : traders
4 The next word of plate is : and
5 The next word of futures is : program
6 The next word of capital is : on
7 The next word of the is : s
8 The next word of it is : freeway
9 The next word of banks is : and
10 The next word of is is : said
11 The next word of their is : the
12 The next word of in is : &
13 The next word of for is : to
14 The next word of were is : its
15 The next word of only is : N
16 The next word of the is : s
17 The next word of extremely is : high
18 The next word of placed is : directly
19 The next word of big is : board
20 The next word of trading is : scandal
21 The next word of the is : s
22 The next word of after is : of
23 The next word of news is : and
24 The next word of rid is : capital
25 The next word of declared is : a
26 The next word of stocks is : in
27 The next word of was is : treatment
28 The next word of which is : comes
29 The next word of